In [1]:
import os
import xarray as xr
import geopandas as gpd
import rioxarray as rxr
from tqdm import tqdm

In [2]:
base_dir = "D:\Maple\Articule_Maple"
nc_dir = os.path.join(base_dir, "futuros_nc")
clip_dir = os.path.join(base_dir, "futuros_clip")
shapefile_path = "data/Politic_divition/Total_quebec_bbox.shp"

In [3]:
os.makedirs(clip_dir, exist_ok=True)

In [4]:
quebec_gdf = gpd.read_file(shapefile_path)
if quebec_gdf.crs is None:
    quebec_gdf = quebec_gdf.set_crs("EPSG:4326")
else:
    quebec_gdf = quebec_gdf.to_crs("EPSG:4326")



In [5]:
quebec_gdf = gpd.read_file(shapefile_path)
if quebec_gdf.crs is None:
    quebec_gdf = quebec_gdf.set_crs("EPSG:4326")
else:
    quebec_gdf = quebec_gdf.to_crs("EPSG:4326")

def open_nc_with_fallback(path):
    # Try preferred engines in order
    for eng in ("netcdf4", "h5netcdf", "scipy"):
        try:
            return xr.open_dataset(path, engine=eng)
        except Exception:
            continue
    raise RuntimeError("No compatible NetCDF engine could open this file.")

def get_spatial_da(ds):
    """
    Pick the first data_var that looks 2D on a lat/lon or y/x grid.
    Return (DataArray, x_name, y_name).
    """
    for var in ds.data_vars:
        dims = tuple(ds[var].dims)
        if set(("lat","lon")).issubset(dims):
            return ds[var], "lon", "lat"
        if set(("y","x")).issubset(dims):
            return ds[var], "x", "y"
    raise ValueError("No variable with spatial dims ('lat','lon') or ('y','x').")

for fname in tqdm(sorted(os.listdir(nc_dir)), desc="Clipping .nc"):
    if not fname.lower().endswith(".nc"):
        continue
    path = os.path.join(nc_dir, fname)
    try:
        print(f"\n📦 Processing {fname} …")
        ds = open_nc_with_fallback(path)
        da, xdim, ydim = get_spatial_da(ds)
        da = da.squeeze()

        # Teach rioxarray which dims are spatial + write CRS
        da = da.rio.set_spatial_dims(x_dim=xdim, y_dim=ydim, inplace=False)

        # If your files are geographic lat/lon, use WGS84. If projected, set that CRS instead.
        # Many future climate NetCDFs are lat/lon WGS84 (no explicit grid_mapping var).
        da = da.rio.write_crs("EPSG:4326", inplace=False)

        # Clip to Québec AOI
        clipped = da.rio.clip(quebec_gdf.geometry, quebec_gdf.crs, drop=True)

        # Save as GeoTIFF
        out_path = os.path.join(clip_dir, fname.replace(".nc", ".tif"))
        clipped.rio.to_raster(out_path)
        ds.close()
        print(f"✅ Saved: {out_path}")

    except Exception as e:
        print(f"❌ Error with {fname}: {e}")
        try:
            ds.close()
        except Exception:
            pass

Clipping .nc:   1%|          | 4/408 [00:00<00:10, 37.95it/s]


📦 Processing BIO10_CMCC_85_2040_79.nc …
✅ Saved: D:\Maple\Articule_Maple\futuros_clip\BIO10_CMCC_85_2040_79.tif

📦 Processing BIO10_CMCC_85_2060_99.nc …
✅ Saved: D:\Maple\Articule_Maple\futuros_clip\BIO10_CMCC_85_2060_99.tif

📦 Processing BIO10_GFDL_45_2040_79.nc …
✅ Saved: D:\Maple\Articule_Maple\futuros_clip\BIO10_GFDL_45_2040_79.tif

📦 Processing BIO10_GFDL_45_2060_99.nc …
✅ Saved: D:\Maple\Articule_Maple\futuros_clip\BIO10_GFDL_45_2060_99.tif

📦 Processing BIO10_GFDL_85_2040_79.nc …
✅ Saved: D:\Maple\Articule_Maple\futuros_clip\BIO10_GFDL_85_2040_79.tif

📦 Processing BIO10_GFDL_85_2060_99.nc …
✅ Saved: D:\Maple\Articule_Maple\futuros_clip\BIO10_GFDL_85_2060_99.tif

📦 Processing BIO10_HADGEM_45_2040_79.nc …
✅ Saved: D:\Maple\Articule_Maple\futuros_clip\BIO10_HADGEM_45_2040_79.tif

📦 Processing BIO10_HADGEM_45_2060_99.nc …
✅ Saved: D:\Maple\Articule_Maple\futuros_clip\BIO10_HADGEM_45_2060_99.tif

📦 Processing BIO10_HADGEM_85_2040_79.nc …
✅ Saved: D:\Maple\Articule_Maple\futuros_clip

Clipping .nc:   4%|▍         | 16/408 [00:00<00:08, 48.42it/s]

✅ Saved: D:\Maple\Articule_Maple\futuros_clip\BIO10_HADGEM_85_2060_99.tif

📦 Processing BIO10_HIST_1960_99.nc …
✅ Saved: D:\Maple\Articule_Maple\futuros_clip\BIO10_HIST_1960_99.tif

📦 Processing BIO10_IPSL_45_2040_79.nc …
✅ Saved: D:\Maple\Articule_Maple\futuros_clip\BIO10_IPSL_45_2040_79.tif

📦 Processing BIO10_IPSL_45_2060_99.nc …
✅ Saved: D:\Maple\Articule_Maple\futuros_clip\BIO10_IPSL_45_2060_99.tif

📦 Processing BIO10_IPSL_85_2040_79.nc …
✅ Saved: D:\Maple\Articule_Maple\futuros_clip\BIO10_IPSL_85_2040_79.tif

📦 Processing BIO10_IPSL_85_2060_99.nc …
✅ Saved: D:\Maple\Articule_Maple\futuros_clip\BIO10_IPSL_85_2060_99.tif

📦 Processing BIO10_MIROC_45_2040_79.nc …
✅ Saved: D:\Maple\Articule_Maple\futuros_clip\BIO10_MIROC_45_2040_79.tif

📦 Processing BIO10_MIROC_45_2060_99.nc …
✅ Saved: D:\Maple\Articule_Maple\futuros_clip\BIO10_MIROC_45_2060_99.tif

📦 Processing BIO10_MIROC_85_2040_79.nc …
✅ Saved: D:\Maple\Articule_Maple\futuros_clip\BIO10_MIROC_85_2040_79.tif

📦 Processing BIO10_MI

Clipping .nc:   6%|▋         | 26/408 [00:00<00:08, 45.31it/s]

...

✅ Saved: D:\Maple\Articule_Maple\futuros_clip\BIO8_NORESM_45_2060_99.tif

📦 Processing BIO8_NORESM_85_2040_79.nc …
✅ Saved: D:\Maple\Articule_Maple\futuros_clip\BIO8_NORESM_85_2040_79.tif

📦 Processing BIO8_NORESM_85_2060_99.nc …
✅ Saved: D:\Maple\Articule_Maple\futuros_clip\BIO8_NORESM_85_2060_99.tif

📦 Processing BIO9_CMCC_85_2040_79.nc …
✅ Saved: D:\Maple\Articule_Maple\futuros_clip\BIO9_CMCC_85_2040_79.tif

📦 Processing BIO9_CMCC_85_2060_99.nc …
✅ Saved: D:\Maple\Articule_Maple\futuros_clip\BIO9_CMCC_85_2060_99.tif

📦 Processing BIO9_GFDL_45_2040_79.nc …
✅ Saved: D:\Maple\Articule_Maple\futuros_clip\BIO9_GFDL_45_2040_79.tif

📦 Processing BIO9_GFDL_45_2060_99.nc …
✅ Saved: D:\Maple\Articule_Maple\futuros_clip\BIO9_GFDL_45_2060_99.tif

📦 Processing BIO9_GFDL_85_2040_79.nc …
✅ Saved: D:\Maple\Articule_Maple\futuros_clip\BIO9_GFDL_85_2040_79.tif

📦 Processing BIO9_GFDL_85_2060_99.nc …
✅ Saved: D:\Maple\Articule_Maple\futuros_clip\BIO9_GFDL_85_2060_99.tif

📦 Processing BIO9_HADGEM_45_20

Clipping .nc:  98%|█████████▊| 398/408 [00:07<00:00, 51.30it/s]


📦 Processing BIO9_HADGEM_45_2060_99.nc …
✅ Saved: D:\Maple\Articule_Maple\futuros_clip\BIO9_HADGEM_45_2060_99.tif

📦 Processing BIO9_HADGEM_85_2040_79.nc …
✅ Saved: D:\Maple\Articule_Maple\futuros_clip\BIO9_HADGEM_85_2040_79.tif

📦 Processing BIO9_HADGEM_85_2060_99.nc …
✅ Saved: D:\Maple\Articule_Maple\futuros_clip\BIO9_HADGEM_85_2060_99.tif

📦 Processing BIO9_HIST_1960_99.nc …
✅ Saved: D:\Maple\Articule_Maple\futuros_clip\BIO9_HIST_1960_99.tif

📦 Processing BIO9_IPSL_45_2040_79.nc …
✅ Saved: D:\Maple\Articule_Maple\futuros_clip\BIO9_IPSL_45_2040_79.tif

📦 Processing BIO9_IPSL_45_2060_99.nc …
✅ Saved: D:\Maple\Articule_Maple\futuros_clip\BIO9_IPSL_45_2060_99.tif

📦 Processing BIO9_IPSL_85_2040_79.nc …
✅ Saved: D:\Maple\Articule_Maple\futuros_clip\BIO9_IPSL_85_2040_79.tif

📦 Processing BIO9_IPSL_85_2060_99.nc …
✅ Saved: D:\Maple\Articule_Maple\futuros_clip\BIO9_IPSL_85_2060_99.tif

📦 Processing BIO9_MIROC_45_2040_79.nc …
✅ Saved: D:\Maple\Articule_Maple\futuros_clip\BIO9_MIROC_45_2040_

Clipping .nc: 100%|██████████| 408/408 [00:07<00:00, 55.16it/s]

✅ Saved: D:\Maple\Articule_Maple\futuros_clip\BIO9_MIROC_85_2040_79.tif

📦 Processing BIO9_MIROC_85_2060_99.nc …
✅ Saved: D:\Maple\Articule_Maple\futuros_clip\BIO9_MIROC_85_2060_99.tif

📦 Processing BIO9_NORESM_45_2040_79.nc …
✅ Saved: D:\Maple\Articule_Maple\futuros_clip\BIO9_NORESM_45_2040_79.tif

📦 Processing BIO9_NORESM_45_2060_99.nc …
✅ Saved: D:\Maple\Articule_Maple\futuros_clip\BIO9_NORESM_45_2060_99.tif

📦 Processing BIO9_NORESM_85_2040_79.nc …
✅ Saved: D:\Maple\Articule_Maple\futuros_clip\BIO9_NORESM_85_2040_79.tif

📦 Processing BIO9_NORESM_85_2060_99.nc …
✅ Saved: D:\Maple\Articule_Maple\futuros_clip\BIO9_NORESM_85_2060_99.tif
